In [ ]:
import pandas as pd
import json
import requests

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Call in terminal:<br>
ollama serve &<br>
ollama pull llama3.1:8b

In [ ]:
IP_ADDRESS = "localhost"
PORT = 11434
ENDPOINT = "/api/chat"
URL = f"http://{IP_ADDRESS}:{PORT}{ENDPOINT}"
HEADERS = {"Content-Type": "application/json"}

MODEL = "llama3.1:8b"

In [ ]:
def generate_binary_inference(url, headers, model, content, retry_count=0, retry_limit=3, timeout=1000):
    try:
        response = requests.post(
        timeout=timeout,
        url=url,
        headers=headers,
        data=json.dumps({
            "model": model,
            "options": {"seed": 351, "temperature": 0, "num_ctx": 16384},
            "stream": False,
            "messages": [
                {
                    "role": "system",
                    "content": "You are an expert at prescribing medicine. You suggest a treatment plan for the given <Patient> based on similar historical cases <Context>. Indicate 'is_opioid: true' if the <Patient> needs an opioid painkiller and 'is_opioid: false' otherwise. Indicate 'is_nonopioid: true' if the <Patient> needs a non-opioid painkiller and 'is_nonopioid: false' otherwise.",
                },
                {
                    "role": "user",
                    "content": content
                }
            ],
            "format": {
                "type": "object",
                "properties": {
                    "is_opioid": {"type": "boolean"},
                    "is_nonopioid": {"type": "boolean"}
                },
                "required": ["is_opioid", "is_nonopioid"]
            }
        }))
        predictions_json_string = response.json()["message"]["content"]
        return json.loads(predictions_json_string)

    except requests.Timeout:
        print(f"{retry_count} | Timed out")
        if retry_count <= retry_limit: return generate_binary_inference(url, headers, model, content, retry_count + 1)

    except requests.JSONDecodeError:
        print(response.reason)
        print(response.text)

In [ ]:
queries = pd.read_csv("/content/drive/MyDrive/queries.csv")

In [ ]:
all_predictions = {"is_opioid": [], "is_nonopioid": []}

for i, row in queries.iterrows():
    predictions = generate_binary_inference(url=URL, headers=HEADERS, model=MODEL, content=row["query"])

    all_predictions["is_opioid"].append(predictions["is_opioid"])
    all_predictions["is_nonopioid"].append(predictions["is_nonopioid"])
    print(f"Row index {i}: Predicted: {predictions['is_opioid'], predictions['is_nonopioid']}, Actual: {row['is_opioid'], row['is_nonopioid']}")

Row index 0: Predicted: (False, True), Actual: (False, False)
Row index 1: Predicted: (False, False), Actual: (False, False)
Row index 2: Predicted: (False, True), Actual: (False, False)
Row index 3: Predicted: (False, True), Actual: (False, False)
Row index 4: Predicted: (True, False), Actual: (True, True)
Row index 5: Predicted: (True, False), Actual: (False, True)
Row index 6: Predicted: (True, True), Actual: (True, True)
Row index 7: Predicted: (False, True), Actual: (False, False)
Row index 8: Predicted: (True, True), Actual: (True, True)
Row index 9: Predicted: (True, True), Actual: (True, True)
Row index 10: Predicted: (False, True), Actual: (False, False)
Row index 11: Predicted: (False, True), Actual: (False, False)
Row index 12: Predicted: (True, False), Actual: (False, False)
Row index 13: Predicted: (True, False), Actual: (True, True)
Row index 14: Predicted: (False, False), Actual: (False, False)
Row index 15: Predicted: (False, True), Actual: (False, True)
Row index 16: P

In [ ]:
pd.DataFrame(all_predictions).to_csv("/content/drive/MyDrive/llm-rag_predictions.csv", index=False)